<a href="https://colab.research.google.com/github/MudSnail/Land_Cover_Classification/blob/main/Basic_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import base libraries
import pandas as pd
import numpy as np
import copy
import joblib
import cloudpickle

#Sklearn - Pipelines, Training
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

#Classification Models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Model metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score

#Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#misc
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Input Data, Combine and Select

In [3]:
#Read in CSVs
james_bay = pd.read_csv('/content/drive/MyDrive/james_bay_data.csv')
labrador = pd.read_csv('/content/drive/MyDrive/labrador_data.csv')
simcoe = pd.read_csv('/content/drive/MyDrive/simcoe_york_data.csv')

In [4]:
#List dataframes to add to james_bay
dfs = [labrador, simcoe]

#Merged data
merged_data = james_bay.append(dfs)

#Print shape and check new data
print(merged_data.shape)
merged_data.head()

(8917500, 13)


,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0055,0.0017,0.0009,0.0016,0.0026,0.0056,0.0001,0.0009,0.0003,0.0032,0.0012,0.0114,0
1,0.0054,0.0020,0.0009,0.0017,0.0021,0.0060,0.0001,0.0005,0.0007,0.0028,0.0014,0.0115,0
2,0.0058,0.0018,0.0007,0.0016,0.0028,0.0060,0.0001,0.0008,0.0007,0.0030,0.0012,0.0124,0
3,0.0057,0.0016,0.0007,0.0016,0.0023,0.0058,0.0001,0.0012,0.0006,0.0031,0.0011,0.0119,0
4,0.0066,0.0013,0.0008,0.0020,0.0029,0.0059,0.0001,0.0008,0.0005,0.0030,0.0014,0.0115,0


In [5]:
#Check classifications values
print(merged_data.Class.value_counts())

15    2836714
18    1050988
1      677877
5      666952
16     646870
13     612009
8      575175
17     512930
12     415493
6      378274
2      151560
11     135890
14      94850
10      86338
19      74619
0         961
Name: Class, dtype: int64


In [6]:
#Drop Class = 0 (null values, which equate to 18)
merged_data = merged_data[merged_data.Class != 0]

#Double check value counts
merged_data.Class.value_counts()

15    2836714
18    1050988
1      677877
5      666952
16     646870
13     612009
8      575175
17     512930
12     415493
6      378274
2      151560
11     135890
14      94850
10      86338
19      74619
Name: Class, dtype: int64

In [7]:
#SubSample ~25 000 for each class
sample = merged_data.groupby('Class').apply(lambda x: x.sample(n=25000)).reset_index(drop=True)
sample.head(4)

,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0102,0.1393,0.1208,0.1634,0.0812,0.0324,0.2017,0.0617,0.1982,0.0351,0.1846,0.0193,1
1,0.0098,0.1121,0.0939,0.1311,0.0539,0.0261,0.1473,0.0513,0.1480,0.0209,0.1397,0.0144,1
2,0.0071,0.0733,0.0501,0.0818,0.0476,0.0276,0.0907,0.0274,0.0844,0.0249,0.0827,0.0168,1
3,0.0268,0.1769,0.1773,0.2008,0.1029,0.0476,0.2199,0.1031,0.2207,0.0484,0.2086,0.0337,1


# Split data

In [8]:
#Set x and y
y = sample['Class']
X = sample.drop('Class', axis = 1)

#Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# Create Pipelines

In [9]:
#Functions for Pipeline
def add_layers(df):
  """
  This function takes in a dataframe and calculates the NDVI, Moisture Index, NDWI and NDSI
  Outputs = dataframe with added layer columns 
  """
  #Create NDVI column (B08-B04)/(B08+B04)
  df['NDVI'] = (df.B08 - df.B04)/(df.B08 + df.B04)
  #Create Moisture index (B8A-B11)/(B8A+B11)
  df['Moisture'] = (df.B8A - df.B11)/(df.B8A + df.B11)
  #Create NDWI (B3-B8)/(B3+B8)
  df['NDWI'] = (df.B03 - df.B08)/(df.B03 + df.B08)
  #create NDSI (B3-B11)/(B3+B11)
  df['NDSI'] = (df.B03 - df.B11)/(df.B03 + df.B11)

  return df

def replace_values(df):
  """
  This function replaces the infinity values with Nan then replaces that with new infinity values
  """
  #Replace infinity values with Nan
  df.replace([np.inf, -np.inf], np.NAN, inplace=True)

  #Fill in null values
  df.fillna(999, inplace=True)

  return df


#Define object transformer class
class objectTransformer:
    """
    Class object transformer, takes in function to apply in preprocessing step of Pipeline
    """
    def __init__(self, func):
        self.func = func
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **fit_params):
        return self.func(X)

In [10]:
#Create Pre-Processing Pipeline
preprocess = Pipeline([("add_layers", objectTransformer(add_layers)),
                       ("replace_values", objectTransformer(replace_values))])

#Create Feature Union
feature_union = FeatureUnion([("PCA", PCA(n_components=3)),
                              ("kKBest", SelectKBest(k=3))])

In [11]:
#Define pipelines
model_1 = Pipeline([('preprocessing', preprocess),
                    ('RFC', RandomForestClassifier(random_state = 123))])

model_2 = Pipeline([('preprocessing', preprocess),
                    ('features', feature_union),
                    ('RFC', RandomForestClassifier(random_state = 123))])

In [12]:
#fit pipes to check which is best
pipes = [model_1, model_2]

for pipe in pipes:
  pipe.fit(X_train, y_train)

#compare accuracies 

#make dictionary
pipe_dict = {0: 'No Feature Union', 1: 'With Feature Union'}

#create for loop to find scores
for i, model in enumerate(pipes):
    print('{} Test Accuracy: {}'.format(pipe_dict[i], model.score(X_test, y_test)))

No Feature Union Test Accuracy: 0.6142133333333333
With Feature Union Test Accuracy: 0.5788622222222222
